#Author - Wonho Lim
#-mail

#Utilize BS4 and Selenium for Crwaling

import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 

In [ ]:
#Utilize BS4 and Selenium for Crwaling

import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 

In [2]:
#Use chromedriver for automated crawling - address can be modified
path = 'C:\\Users\\DT\\Desktop\\자료\\chromedriver'

In [3]:
#Basic setting for csv file that will store 재무정보 data - Order of rows are based on 재무정보 데이터소스
cover = open('DutchData.csv','w', newline='', encoding='CP949')
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드', '현지언어국가명','영문국가명','국가언어코드값','현지언어명','영문언어명','현지언어기업명','통화구분코드','화폐단위명','영문기업명','법인등록번호','사업자등록번호','현지언어산업군명','영문산업군명','설립일자','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','회계연도','회계분기','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','이자비용','기타영업외비용금액','영업이익손실금액','이자수익','기타영업외수익금액','법인세차감전순이익','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업CAGR','기업 연수','기업 상태','기업추가정보값','작업구분코드','데이터생성일자','연계처리상태코드','연계처리일자'])

433

In [4]:
#Automatic access to netherlands companies list from EURONEXT
base_url = "https://live.euronext.com/"
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/products/equities/list')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#popup-buttons > button.agree-button.eu-cookie-compliance-default-button').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, value='#div-stocks-data-table > button').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, value='#collapsePanelCompactGroupheadingitem1').click()
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, value='#collapsePanelCompactGroup > div:nth-child(1) > div.card-header.panels__header.panels__header--space-between.m-0.bg-transparent > div.panels__column.ml-auto > button > div').click()
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, value='#edit-market-06--2 > div > label').click()
time.sleep(5)
driver.find_element(By.CSS_SELECTOR, value='#edit-awl-pd-filters-submit--2').click()

#Get each company's link on table - P1
time.sleep(4)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
time.sleep(4)
all_atag_maintable = maintable.find_all('a')


#Get each company's link on table - P2
driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(4)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable2 = soup.find('table', id='stocks-data-table')
time.sleep(4)
all_atag_maintable2 = maintable2.find_all('a')

#Merge two link lists
all_atag_maintable = all_atag_maintable + all_atag_maintable2

company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)

# To check if all companies are selected
#print(len(company_links))

#Main Data Collection
for sub in company_links:
    driver.get(base_url + sub)
    time.sleep(5)
    info = []
    
    info.append("NLD")
    info.append("Nederland")
    info.append("Nederlands")
    info.append("NL")
    info.append("Nederlands")
    info.append("Dutch")
    info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text)
    info.append("EUR")
    info.append("유로")
    info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text)
    info.append("(NA)법인등록번호")
    info.append("(NA)사업자등록번호")
    
    driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
    time.sleep(5)
    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2)').text)
    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2)').text)
    info.append("(NA) 설립일자")
    info.append("(NA) 시장코드")
    info.append("EURONEXT AMSTERDAM")
    info.append("EURONEXT AMSTERDAM")
    info.append("(NA) 상장코드")
                
    info.append("NA 상장일자")
    driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
    time.sleep(5)
    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[12]/td').text)
    info.append("(NA) 회계분기")
    info.append("(NA) 유동자산금액")
    info.append("(NA) 비유동자산금액")
    info.append("(NA) 자산총계금액")
    info.append("(NA) 유동부채금액")
    info.append("(NA) 비유동부채금액")
    info.append("(NA) 부채총계금액")
    info.append("(NA) 자본총계금액")
    info.append("(NA) 부채자본총계금액")
    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "천 유로")
    info.append("(NA) 매출원가금액")
    info.append("(NA) 판매비관리비금액")
    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]').text + "천 유로")
    info.append("(NA) 기타영업외비용금액")
    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]').text + "천 유로")
    info.append("(NA) 이자수입")
    info.append("(NA) 기타영업외수익금액")
    
    info.append("(계산예정)법인세차감전순이익")
    
    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]').text + "천 유로")
    info.append("(NA) 영업활동현금흐름금액")
    info.append("(NA) 투자활동현금흐름금액")
    info.append("(NA) 재무활동현금흐름금액")
    info.append("(NA) 기초현금자산금액")
    info.append("(NA) 기말현금자산금액")
    info.append("(NA) 부채비율")
                
    info.append("(계산예정)영업이익율")
    info.append("(계산예정)매출액증가율")
    info.append("(계산예정)영업이익증가율")
    info.append("(계산예정)당기순이익 증가율")
    info.append("(계산예정)기업CAGR")
    info.append("(계산예정)기업 연수")        
    info.append("(계산예정)기업 상태")

    driver.find_element(By.CSS_SELECTOR, value='#awl-instrument-page-quicktabs-empty > div > div > nav > div > a.nav-item.nav-link.active').click()
    time.sleep(5)
    info.append(info.append("주당 가치: " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[2]/div/div[1]/div[1]/div').text))
    info.append("(NA) 작업구분코드")
    info.append(info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[2]/div/div[2]/div[2]/div[1]/span').text))
    info.append("(NA) 연계처리상태코드")
    info.append("(NA) 연계처리일자")
    time.sleep(1)
    
    writing.writerow(info);
    print(info)

C:\Users\DT\AppData\Local\Temp\ipykernel_13100\3367449046.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


188
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'AALBERTS N.V.', 'EUR', '유로', 'AALBERTS N.V.', '(NA)법인등록번호', '(NA)사업자등록번호', '5020, Industrial Goods and Services', '5020, Industrial Goods and Services', '(NA) 설립일자', '(NA) 시장코드', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', '(NA) 상장코드', 'NA 상장일자', 'Source: Cofisem - Last Update: 22 Apr 2022', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '2.979.100천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '13.400천 유로', '(NA) 기타영업외비용금액', '506.100천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '367.800천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당 가치: € 45.28', None, '(NA) 작업구분코드', '17/05/2022 - 17:40  CET', None, '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland'

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'AIR FRANCE -KLM', 'EUR', '유로', 'AIR FRANCE -KLM', '(NA)법인등록번호', '(NA)사업자등록번호', '4050, Travel and Leisure', '4050, Travel and Leisure', '(NA) 설립일자', '(NA) 시장코드', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', '(NA) 상장코드', 'NA 상장일자', 'Source: Cofisem - Last Update: 17 May 2022', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '14.315.000천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '728.000천 유로', '(NA) 기타영업외비용금액', '-2.532.000천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '-3.294.000천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당 가치: € 3.962', None, '(NA) 작업구분코드', '17/05/2022 - 17:40  CET', None, '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands'

['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 'APERAM', 'EUR', '유로', 'APERAM', '(NA)법인등록번호', '(NA)사업자등록번호', '5510, Basic Resources', '5510, Basic Resources', '(NA) 설립일자', '(NA) 시장코드', 'EURONEXT AMSTERDAM', 'EURONEXT AMSTERDAM', '(NA) 상장코드', 'NA 상장일자', 'Source: Cofisem - Last Update: 16 May 2022', '(NA) 회계분기', '(NA) 유동자산금액', '(NA) 비유동자산금액', '(NA) 자산총계금액', '(NA) 유동부채금액', '(NA) 비유동부채금액', '(NA) 부채총계금액', '(NA) 자본총계금액', '(NA) 부채자본총계금액', '5.102.000천 유로', '(NA) 매출원가금액', '(NA) 판매비관리비금액', '4.000천 유로', '(NA) 기타영업외비용금액', '1.042.000천 유로', '(NA) 이자수입', '(NA) 기타영업외수익금액', '(계산예정)법인세차감전순이익', '969.000천 유로', '(NA) 영업활동현금흐름금액', '(NA) 투자활동현금흐름금액', '(NA) 재무활동현금흐름금액', '(NA) 기초현금자산금액', '(NA) 기말현금자산금액', '(NA) 부채비율', '(계산예정)영업이익율', '(계산예정)매출액증가율', '(계산예정)영업이익증가율', '(계산예정)당기순이익 증가율', '(계산예정)기업CAGR', '(계산예정)기업 연수', '(계산예정)기업 상태', '주당 가치: € 35.64', None, '(NA) 작업구분코드', '17/05/2022 - 17:40  CET', None, '(NA) 연계처리상태코드', '(NA) 연계처리일자']
['NLD', 'Nederland', 'Nederlands', 'NL', 'Nederlands', 'Dutch', 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[13]/td"}
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BAB8F3+2406643]
	Ordinal0 [0x00B3AF31+1945393]
	Ordinal0 [0x00A2C748+837448]
	Ordinal0 [0x00A592E0+1020640]
	Ordinal0 [0x00A5957B+1021307]
	Ordinal0 [0x00A86372+1205106]
	Ordinal0 [0x00A742C4+1131204]
	Ordinal0 [0x00A84682+1197698]
	Ordinal0 [0x00A74096+1130646]
	Ordinal0 [0x00A4E636+976438]
	Ordinal0 [0x00A4F546+980294]
	GetHandleVerifier [0x00E19612+2498066]
	GetHandleVerifier [0x00E0C920+2445600]
	GetHandleVerifier [0x00C44F2A+579370]
	GetHandleVerifier [0x00C43D36+574774]
	Ordinal0 [0x00B41C0B+1973259]
	Ordinal0 [0x00B46688+1992328]
	Ordinal0 [0x00B46775+1992565]
	Ordinal0 [0x00B4F8D1+2029777]
	BaseThreadInitThunk [0x773FFA29+25]
	RtlGetAppContainerNamedObjectPath [0x775F7A